# 02- Exploratory Data Analysis (EDA)

In [1]:
import pickle
import pandas as pd
import numpy as np

from gensim.models import Word2Vec

In [2]:
df = pd.read_pickle('../data/cleaned_tweets.pkl')

In [3]:
bow = []
for word in df.cleaned.str.split().to_list():
    bow += word

In [4]:
bow_set = set(bow)

In [5]:
len(bow_set)

9861

In [6]:
texts = [(tweet, cat) for tweet, cat in zip(df.cleaned, df.category.to_list())]

In [7]:
tweets, cats = (zip(*texts))

In [8]:
corpus = [tweet.split()for tweet in tweets]

In [9]:
model_cbow = Word2Vec(corpus, vector_size=300, min_count=1, epochs=10, seed=42)
model_sg = Word2Vec(corpus, sg=1, vector_size=300, min_count=1, epochs=10, seed=42)

In [10]:
we_cbow = {word: model_cbow.wv[word] for word in bow_set}
we_sg = {word: model_sg.wv[word] for word in bow_set}

In [11]:
len(we_cbow), len(we_sg)

(9861, 9861)

In [12]:
def meaner(word_embedding, tweets):
    tweet_embedding = {}
    for i, tweet in enumerate(tweets):
        tweet_embedding[tweet] = np.mean(np.array([word_embedding[word] for word in tweets[i].split()]), axis=0)
    return tweet_embedding

In [13]:
te_cbow = meaner(we_cbow, tweets)
te_sg = meaner(we_sg, tweets)

In [14]:
te_cbow_df = pd.DataFrame(te_cbow).T.reset_index().rename(columns={'index':'tweet'})
te_cbow_df['category'] = cats
te_cbow_df['label'] = te_cbow_df.category.map({'unreliable':0, 'true':1})
te_cbow_df = te_cbow_df.sample(frac=1, random_state=42).reset_index(drop=True)

te_sg_df = pd.DataFrame(te_sg).T.reset_index().rename(columns={'index':'tweet'})
te_sg_df['category'] = cats
te_sg_df['label'] = te_sg_df.category.map({'unreliable':0, 'true':1})
te_sg_df = te_sg_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
pickle.dump(te_cbow_df, open("../data/te_cbow_df_training_unshared.pickle", "wb"))
pickle.dump(te_sg_df, open("../data/te_sg_df_training_unshared.pickle", "wb"))

In [16]:
pickle.dump(we_cbow, open("../data/we_cbow_training_unshared.pickle", "wb"))
pickle.dump(we_sg, open("../data/we_sg_training_unshared.pickle", "wb"))